<a href="https://colab.research.google.com/github/rahgoar/dementia_eHealthcare/blob/main/Job_assignment_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A bipartite graph is a graph whose vertices can be divided into two disjoint sets U and V such that every edge connects a vertex in U to a vertex in V. The assignment problem is to find a perfect matching in a bipartite graph, where each vertex in U is matched to exactly one vertex in V.

In [ ]:

# Python program to find
# maximal Bipartite matching.

class GFG:
    def __init__(self,graph):

        # residual graph
        self.graph = graph
        self.ppl = len(graph)
        self.jobs = len(graph[0])

    # A DFS based recursive function
    # that returns true if a matching
    # for vertex u is possible
    def bpm(self, u, matchR, seen):

        # Try every job one by one
        for v in range(self.jobs):

            # If applicant u is interested
            # in job v and v is not seen
            if self.graph[u][v] and seen[v] == False:

                # Mark v as visited
                seen[v] = True

                '''If job 'v' is not assigned to
                an applicant OR previously assigned
                   applicant for job v (which is matchR[v])
                   has an alternate job available.
                   Since v is marked as visited in the
                   above line, matchR[v]  in the following
                   recursive call will not get job 'v' again'''
                if matchR[v] == -1 or self.bpm(matchR[v],
                                               matchR, seen):
                    matchR[v] = u
                    return True
        return False

    # Returns maximum number of matching
    def maxBPM(self):
        '''An array to keep track of the
           applicants assigned to jobs.
           The value of matchR[i] is the
           applicant number assigned to job i,
           the value -1 indicates nobody is assigned.'''
        matchR = [-1] * self.jobs

        # Count of jobs assigned to applicants
        result = 0
        for i in range(self.ppl):

            # Mark all jobs as not seen for next applicant.
            seen = [False] * self.jobs

            # Find if the applicant 'u' can get a job
            if self.bpm(i, matchR, seen):
                result += 1
        return result
bpGraph =[[0, 1, 1, 0, 0, 0],
          [1, 0, 0, 1, 0, 0],
          [0, 0, 1, 0, 0, 0],
          [0, 0, 1, 1, 0, 0],
          [0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 1]]

g = GFG(bpGraph)

print ("Maximum number of applicants that can get job is %d " % g.maxBPM())

# This code is contributed by Neelam Yadav

In [ ]:
# greedy search to find the best (Max, Min) correspondence based on the associated Cohen Kappa.
def best_kappa_mapping(df):
    book_numbers = sorted(df['actual_labels'].unique())
    cluster_numbers = sorted(df['cluster_labels'].unique())

    # Ensure there is a valid mapping
    assert len(book_numbers) == len(cluster_numbers), "Number of unique book numbers and cluster numbers must be the same"

    best_kappa = -1
    best_mapping = {}

    # Iterate through all possible permutations of book numbers
    for perm in permutations(book_numbers):
        cluster_mapping = {cluster: book for cluster, book in zip(cluster_numbers, perm)}
        # Map clusters to book numbers based on current permutation
        mapped_clusters = df['cluster'].map(cluster_mapping)
        # Calculate Cohen's Kappa score
        kappa = cohen_kappa_score(df['book_number'], mapped_clusters)
        #print(kappa)
        # Update best mapping if this permutation has the highest Kappa score
        if kappa > best_kappa:
            best_kappa = kappa
            best_mapping = cluster_mapping
    print(f"best kappa: {best_kappa}")
    print(f"best mapping: {best_mapping}")
    return best_mapping

In [ ]:
# Best mapping based on the optimized reward. Optimization formulation.
from pulp import LpProblem,LpVariable,LpMaximize,lpSum,LpStatus,value

def mapping_labels(y_pred,y_label):
  M= construct_assignment_matrix(y_pred,y_label)

  prob = LpProblem("Max_Matching", LpMaximize)
  var_name_prefix = 'IsSelected'
  N = 5
  vars = LpVariable.dicts("Choice", (range(N), range(N)), cat="Binary")

  objective_func = lpSum([vars[i][j]*M[i][j] for i in range(N) for j in range(N)]), "Total_correct_classes"
  prob += objective_func
  for i in range(N):
    prob += lpSum([vars[i][j] for j in range(N)]) ==1 , f"row_{i}"

  for j in range(N):
    prob += lpSum([vars[i][j] for i in range(N)]) ==1, f"column_{j}"

  #prob.writeLP("MaximumMatching.lp")

  prob.solve()

  #print("Status:", LpStatus[prob.status])

  cluster_to_class = {}
  for v in prob.variables():
      if v.varValue>0.1: #working with float :)
        _,cluster,class_id = v.name.split('_')
        cluster_to_class[int(cluster)] = int(class_id)
  y_pred_class = np.zeros(y_pred.shape)
  for i,r in enumerate(y_pred):
    y_pred_class[i] = cluster_to_class[r]

  return y_pred_class, cluster_to_class

Limitations: It's primarily suitable for bipartite graphs with complete matchings. For more complex matching problems, other algorithms like the Hopcroft-Karp algorithm might be more appropriate.

1-Cost Matrix: We represent the bipartite graph as a cost matrix, where each element cost_matrix[i, j] represents the cost of assigning the i-th node in one set to the j-th node in the other set.
2-Linear Sum Assignment: The linear_sum_assignment function from scipy.optimize efficiently solves the assignment problem by reducing it to a minimum weight matching problem in a bipartite graph.
3-Output: The function returns two arrays, row_ind and col_ind, indicating the optimal assignment.

In [ ]:
#The Hungarian algorithm, also known as the Munkres algorithm

import numpy as np
from scipy.optimize import linear_sum_assignment

def hungarian_algorithm(cost_matrix):
    """
    Finds the optimal assignment for a bipartite graph using the Hungarian algorithm.

    Args:
        cost_matrix: A 2D numpy array representing the cost matrix.

    Returns:
        A tuple (row_ind, col_ind) representing the optimal assignment.
    """

    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    return row_ind, col_ind

# Example usage:
cost_matrix = np.array([[10, 15, 20],
                        [5, 25, 10],
                        [15, 5, 15]])

row_ind, col_ind = hungarian_algorithm(cost_matrix)

print("Optimal assignment:")
for i, j in zip(row_ind, col_ind):
    print(f"Row {i} is assigned to Column {j} with cost {cost_matrix[i, j]}")